<a href="https://colab.research.google.com/github/mwelland/ENGPYHS_3NM4/blob/main/Initial_value_problems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial value problems

Initial Value Problems are characterised by knowledge of boundary conditions only on one side of the domain. This is typically (but not necessarily) for time-dependent equations, thus the name and following nomenclature.

e.g.:
 $$\frac{dy}{dt} = y, \quad y(t=0) = 1$$
 or
 $$\frac{d^2 y}{dt^2} = -g, \quad y(t=0) = 1000, y'(t=0) = 0$$

In these cases we are forced to start at $t=0$ and *time-step* forward until we reach our goal time. This has several implications:


*   Error in the soultion will accumulate with successive steps.
*   Numerical noise may destabilize the solution scheme.
*   Time stepping must resolve the time-scale of the physics we are looking at.
*   The time-scale may change mid-problem.
*   We will build up a *history* of steps as we go.

As with analytical methods, on a high level we obtain a solution by *integrating* the differential equation up until the point of interest. For this reason, solving IVPs are often called *time integrators* and much of the analysis carries over.

## Error analysis

As usual, we will discuss error in the context of truncation error. As with other integration schemes,
* *Local truncation error* occurs over the time-step due to the approximation of $y_{i+1}$.
* *Global / propogated truncation error* is the accumulation of propogated local truncation error and results in overall error of the approximation to the true solution.

Later we will discuss a new phenomenon, stability and *stiffness*.

We will be using the Taylor expansion to analyse error and help deevelop our numerical schemes. Higher accuracy is achieved through estimation of the higher order derivatives. There are two classes of methods that are applied:
* Single step methods - Uses information about the current time step
* Multistep methods - Uses the history of time steps

##Runge-Kutta methods

The Runge-Kutta methods are a class of techniques where the next time step is calculated from the *current time step* and an estimate of its *slope* (rate of change).

### Forward Euler method

In the Forward Euler method (aka: Euler-Cauchy / point-slope method, Explicit Euler) one simply takes the slope at the current point, which is given in the differential equation, and assumes it is constant over the step:

Given, $$ \frac{dy}{dx} = f(x, y) $$
the next time step is:
$$ y_{i+1} = y_i + f(x_i, y_i) h$$

which is simply the left Riemann sum, and similarly the error is $O(h^2)$ over the step, and $O(h)$ over the full solution.

###Heun's method

Now that we have a value for $y_{i+1}$ is there a way we can use the *prediction* to *correct* itself? Heun's method does exactly this and is called a **predictor-corrector** algorithm as a result.

Consider:

$$ \frac{dy}{dx} = f(x, y) $$
make a prediction,
$$ y^0_{i+1} = y_i + f(x_i, y_i) h$$
Now take the average of the slopes:
$$\begin{align}
\bar{y}' &= \frac{y_i+y_{i+1}}{2} \\
&= \frac{f(x_i,y_i)+f(x_{i+1}, y^0_{i+1})}{2}
\end{align} $$
which is used in the corrector:
$$ y_{i+1} = y_i + \frac{f(x_i,y_i)+f(x_{i+1}, y^0_{i+1})}{2} h$$


This is interesting since one could repeat the predictor-corrector cycle with the intention of converging towards the correct answer, but we will see this is a bad idea:



#### Example: Heun's method with multiple predictor-corrector cycles

Integrate
$$y' = 4 e^{0.8 x}-0.5 y$$
with
$$ y(0)=2$$
using Heun's method with 1 and 15 predictor-corrector cycles.


In [1]:
import numpy as np

def f(x, y):
  return 4 * (2.71828 ** (0.8 * x)) - 0.5 * y

def heuns_method(x0, y0, h, t, iterations):
  """
  Applies Heun's method to approximate the solution of a differential equation.

  Args:
    x0: The initial x value.
    y0: The initial y value.
    h: The step size.
    t: The target time for the approximation.
    iterations: The number of iterations for the predictor-corrector cycle.

  Returns:
    The approximate y value at time t.
  """
  x = x0
  y = y0
  while x < t:
    y_pred = y + h * f(x, y)
    y_next = y + h * (f(x, y) + f(x + h, y_pred)) / 2
    for _ in range(iterations - 1):
      y_pred = y_next
      y_next = y + h * (f(x, y) + f(x + h, y_pred)) / 2

    y = y_next
    x += h
    print("Time ", x, ", approximation, ", y_next, ', True ,', 4/1.3*(np.exp(.8*x)-np.exp(-.5*x))+2*np.exp(-.5*x))
  return y


# Initial conditions
x0 = 0
y0 = 2

# Step size and target time
h = 1
t = 4

# Estimate integral with Heun's method, iterating once
print("Heun's method with 1 iteration")
y_approx_once = heuns_method(x0, y0, h, t, 1)

# Estimate integral with Heun's method, iterating 15 times
print("\nHeun's method with 15 iterations")
y_approx_15 = heuns_method(x0, y0, h, t, 15)

Heun's method with 1 iteration
Time  1 , approximation,  6.701079461759733 , True , 6.194631377209372
Time  2 , approximation,  16.319768581929353 , True , 14.84392190764649
Time  3 , approximation,  37.199199627848756 , True , 33.67717176796817
Time  4 , approximation,  83.33761313495894 , True , 75.33896260915857

Heun's method with 15 iterations
Time  1 , approximation,  6.360863570675189 , True , 6.194631377209372
Time  2 , approximation,  15.302225064771143 , True , 14.84392190764649
Time  3 , approximation,  34.74323213193692 , True , 33.67717176796817
Time  4 , approximation,  77.73495685652544 , True , 75.33896260915857


Unfortunatley, Heun's method does converge but not necessarily to the correct answer!

#### The Trapezoid rule

In the case that the slope doesn't depend on the function value, $y'(x) = f(x)$ the predictor can be calcualted directly, eliminating the cycle:

$$ y_{i+1} = y_i + \frac{f(x_i)+f(x_{i+1})}{2} h$$

which is mearly the trapezoid rule which carries $O(h^3)$ accuracy locally and $O(h^2)$ globally.

### The Midpoint method

Recall from the discussion on on differentiation / integration that information from the midpoint was often superior that of either endpoint (in isolation) as over/under estimates tend to cancel.

As an alternative to Heun's predictor-corrector method, let's subdivide the interval and find the slope at the *midpoint*. To do this, take a half step:

$$ y_{i+1/2} = y_i + f(x_i,y_i) \frac{h}{2} $$

then use the slope at the midpoint, $y'_{i+1/2} = f(x_{i+1/2},y_{i+1/2})$ to estimate the full step:

$$ y_{i+1} = y_i + f(x_{i+1/2},y_{i+1/2}) h $$

which has $O(h^3)$ local / $O(h^2)$ global error.




##Runge-Kutta methods

Runge-Kutta methods achieve the accuracy of a Taylor Series approach without requiring explicit derivatives. They can generally be cast in the form,

$$ y_{i+1} = y_i + \phi(x_i, y_i, h) h $$
where the *increment function* represents the slope over the interval. The function can generally be written,
$$ \phi = a_1 k_1 + a_2 k_2 + a_3 k_3 + ... + a_n k_n $$

with
$$\begin{align}
k_1 &= f(x_i, y_i) \\
k_2 &= f(x_i + p_1 h, y_i + q_{11} k_1 h) \\
k_3 &= f(x_i + p_2 h, y_i + q_{21} k_1 h + q_{22} k_2 h) \\
\vdots \\
k_n &= f(x_i + p_{n-1} h, y_i + q_{n-1,1} k_1 h + q_{n-1,2} k_2 h + ... + q_{n-1,n-1} k_{n-1} h)
\end{align} $$
where the $p$'s and $q$'s are constants and determine the type of RK method.


Note RK-1 is simply the Forward Euler equation.


### Butcher Tableaus

Since there are infinitudes of potential RK methods, mathematicians generalize the process and describe them in a *Butcher Tableau*:

$$y_{i+1} = y_i + h \sum_{i=1}^s a_i k_i$$
with
$$\begin{align}
k_1 &= f(x_i, y_i) \\
k_2 &= f(x_i + p_2 h, y_i + [q_{21}k_1] h) \\
k_3 &= f(x_i + p_3 h, y_i + [q_{31}k_1 + q_{32}k_2] h) \\
\vdots\\
k_s &= f(x_i + p_s h, y_i + [q_{s1}k_1 + q_{s2}k_2 ... q_{s,s-1} k_{s-1}] h) \\
\end{align} $$




The collection of constants is written compactly in a Butcher Tableau:

$$\begin{array}{c|ccccc}
p_1 & & & & &\\
p_2 & q_{21} &  &  &  &\\
p_3 & q_{31} & q_{32} &  &  &\\
\vdots & \vdots &  & \ddots &  &\\
p_s & q_{s1} & q_{s2} & \cdots & q_{s,s-1} &\\
\hline
 & a_1 & a_2 & \cdots & a_{s-1} & a_s \\
\end{array}$$

### RK-2 methods

The second order RK method is:

$$y_{i+1} = y_i + [a_1 k_1 + a_2 k_2] h $$
with
$$\begin{align}
k_1 &= f(x_i, y_i) \\
k_2 &= f(x_i + p_1 h, y_i+q_{11} k_1 h) \end{align} $$

Taylor expanding $k_2$ in $p$ and $q$,
$$ f(x_i + p_1 h, y_i+q_{11} k_1 h) = f(x_i,y_i) + p_1 h \frac{\partial f}{\partial x} + q_{11} k_1 h \frac{\partial f}{\partial y} + O(h^2)$$

which plugged back in to $y_{i+1}$:
$$y_{i+1} = y_i + [a_1+a_2] f(x_i, y_i) h + \bigg[a_2 p_1 \frac{\partial f}{\partial x} + a_2 q_{11} f(x_i, y_i) \frac{\partial f}{\partial y} \bigg] h^2 + O(h^3)   $$

which we can compare to a second order Taylor expansion:
$$ \begin{align}
y_{i+1} &= y_i + f(x_1, y_1) h + \frac{f'(x_1, y_1)}{2} h^2 \\
&= y_i + f(x_1, y_1) h + \bigg[\frac{\partial f}{\partial x} +\frac{\partial f}{\partial x} \frac{dy}{dx} \bigg] \frac{h}{2} \end{align}$$

Comparing terms we see:
$$\begin{align}
a_1+a_2 &= 1 \\
a_2 p_1 &= \frac{1}{2} \\
a_2 q_{11}&= \frac{1}{2}\\
\end{align} $$




Here we see that we have a single degree of freedom for the set of constants! Any choice will satisfy 2'nd order equations and therefor ebe exact for constant, linear, or quadratic ODEs. Certaint choices will have better properties in general.



#### Heun's method

With $a_2 = \frac{1}{2}$,

$$y_{i+1} = y_i + \frac{k_1 + k_2}{2} h $$
with
$$\begin{align}
k_1 &= f(x_i, y_i) \\
k_2 &= f(x_i + h, y_i + k_1 h) \end{align} $$

which is simply Heun's method.

\begin{array}{c|cc}
0 & 0 & 0 \\
1 & 1 & 0 \\
\hline
& \frac{1}{2} & \frac{1}{2} \\
\end{array}

#### Midpoint method


With $a_2 = 1$,
$$y_{i+1} = y_i + k_2 h $$
with
$$\begin{align}
k_1 &= f(x_i, y_i) \\
k_2 &= f(x_i + \frac{1}{2}h, y_i + \frac{1}{2} k_1 h) \end{align} $$
which is the midpoint method.

\begin{array}{c|cc}
0 & 0 & 0 \\
\frac{1}{2} & \frac{1}{2} & 0 \\
\hline
& 0 & 1 \\
\end{array}

#### Ralston's method


The choice $a_2 = \frac{2}{3}$, can be shown to provide a minimum bound on the truncation error,
$$y_{i+1} = y_i + [k_1 + 2 k_2 ] \frac{h}{3} $$
with
$$\begin{align}
k_1 &= f(x_i, y_i) \\
k_2 &= f(x_i + \frac{3}{4}h, y_i + \frac{3}{4} k_1 h) \end{align} $$

\begin{array}{c|cc}
0 & 0 & 0 \\
\frac{2}{3} & \frac{2}{3} & 0 \\
\hline
& \frac{1}{4} & \frac{3}{4} \\
\end{array}

### RK-3 methods

A similar derivation follows for RK3 and RK4 methods, again with choices for the missing degrees of freedom.

Common choices are:
#### RK3
$$y_{i+1} = y_i + \frac{1}{6}[k_1 + 4 k_2 + k_3] h $$
with
$$\begin{align}
k_1 &= f(x_i, y_i) \\
k_2 &= f(x_i + \frac{1}{2}h, y_i + \frac{1}{2} k_1 h) \\
k_3 &= f(x_i + h, y_i - k_1 h + 2 k_2 h) \\
\end{align} $$

which reduces to Simpson's 1/3 Rule if $f$ is only a function of $x$. As with Simpson's rule, it is $O(h^4)$ local / $O(h^3)$ global error.

The Butcher Tableau is:
\begin{array}{c|ccc}
0 &  &  &  \\
\frac{1}{2} & \frac{1}{2} &  &  \\
1 & -1 & 2 &  \\
\hline
& \frac{1}{6} & \frac{2}{3} & \frac{1}{6} \\
\end{array}



#### RK4

$$y_{i+1} = y_i + \frac{1}{6}[k_1 + 2 k_2 + 2 k_3 + k_4] h $$
with
$$\begin{align}
k_1 &= f(x_i, y_i) \\
k_2 &= f(x_i + \frac{1}{2}h, y_i + \frac{1}{2} k_1 h) \\
k_3 &= f(x_i + \frac{1}{2}h, y_i + \frac{1}{2} k_2 h ) \\
k_4 &= f(x_i + h, y_i + k_3 h)
\end{align} $$

with $O(h^5)$ local / $O(h^4)$ global error.

The Butcher Tableau is:
$$
\begin{array}{c|cccc}
0 &  &  &  &  \\
\frac{1}{2} & \frac{1}{2} &  &  &  \\
\frac{1}{2} & 0 & \frac{1}{2} &  &  \\
1 & 0 & 0 & 1 &  \\
\hline
 & \frac{1}{6} & \frac{1}{3} & \frac{1}{3} & \frac{1}{6} \\
\end{array}
$$

#### Example - RK4 steps
For full accuracy, the increment function should be ***, *but we don't have higher derivatives*. Instead we build up the effect by taking partial steps, with our current best guess of the increment function.

Runge-Kutta_slopes.svg